In [138]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sqlalchemy import create_engine
import warnings

pd.options.display.max_columns = 999

In [203]:
warnings.filterwarnings('ignore')
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

useducation_df = pd.read_sql_query('select * from useducation',con=engine)

useducation_df_orignal = useducation_df.copy()

engine.dispose()

In [204]:
n = -1
for values in useducation_df_orignal['OTHER_EXPENDITURE']:
    n += 1
    if math.isnan(values) == True:
        useducation_df_orignal['OTHER_EXPENDITURE'].iloc[n] = useducation_df_orignal['TOTAL_EXPENDITURE'].iloc[n] - useducation_df_orignal['INSTRUCTION_EXPENDITURE'].iloc[n] - useducation_df_orignal['SUPPORT_SERVICES_EXPENDITURE'].iloc[n] - useducation_df_orignal['CAPITAL_OUTLAY_EXPENDITURE'].iloc[n]
        


    
    
years = useducation_df_orignal["YEAR"].unique()
na_columns_list = list(useducation_df_orignal.columns)
del na_columns_list[:3]
for year in years:
    useducation_df_orignal.loc[useducation_df_orignal["YEAR"] == year, na_columns_list] = useducation_df_orignal.loc[useducation_df_orignal["YEAR"] == year, na_columns_list].interpolate()
    
states = useducation_df_orignal["STATE"].unique()
for state in states:
    useducation_df_orignal.loc[useducation_df_orignal["STATE"] == state, 'ENROLL'] = useducation_df_orignal.loc[useducation_df_orignal["STATE"] == state, 'ENROLL'].fillna(useducation_df_orignal.loc[useducation_df_orignal['STATE'] == state, 'ENROLL'].mean())
    
useducation_df_orignal.loc[0, 'AVG_READING_8_SCORE'] = useducation_df_orignal.loc[useducation_df_orignal['YEAR'] == 1992,'AVG_READING_8_SCORE'].mean()




In [205]:
useducation_df_orignal.dropna(inplace=True)
useducation_df_orignal = useducation_df_orignal.reset_index()

## Problem 1

In [206]:
Score_Var = []
for value in list(useducation_df_orignal.index):
    Total_G48 = useducation_df_orignal['GRADES_4_G'].iloc[value] + useducation_df_orignal['GRADES_8_G'].iloc[value]
    Score_Var.append(((useducation_df_orignal['AVG_MATH_4_SCORE'].iloc[value] * useducation_df_orignal['GRADES_4_G'].iloc[value] / Total_G48) 
                      + (useducation_df_orignal['AVG_READING_4_SCORE'].iloc[value] * useducation_df_orignal['GRADES_4_G'].iloc[value] / Total_G48) 
                      + (useducation_df_orignal['AVG_MATH_8_SCORE'].iloc[value] * useducation_df_orignal['GRADES_8_G'].iloc[value] / Total_G48) 
                      + (useducation_df_orignal['AVG_READING_8_SCORE'].iloc[value] * useducation_df_orignal['GRADES_8_G'].iloc[value] / Total_G48))/2)

In [207]:
useducation_df_orignal.insert(25,'Scoring Variable', Score_Var)

In [208]:
useducation_df_orignal.head()

,index,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,GRADES_PK_G,GRADES_KG_G,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,Scoring Variable,AVG_READING_8_SCORE
0,0,1992_ALABAMA,ALABAMA,1992,7.365843e+05,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,735036.0,263006.0,174053.0,8224.0,55460.0,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,232.724444,262.353629
1,1,1992_ALASKA,ALASKA,1992,1.304296e+05,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,350902.0,85773.0,37451.0,2371.0,10152.0,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,211.790904,258.776900,207.088116,232.851618,258.859712
2,2,1992_ARIZONA,ARIZONA,1992,8.743221e+05,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,1007732.0,348826.0,609114.0,2544.0,53497.0,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,235.639070,262.169895
3,3,1992_ARKANSAS,ARKANSAS,1992,4.519019e+05,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,483488.0,149999.0,145212.0,808.0,33511.0,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,235.441283,264.619665
4,4,1992_CALIFORNIA,CALIFORNIA,1992,5.945343e+06,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,8520926.0,2214296.0,2044688.0,59067.0,431763.0,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,230.756386,265.519676


## Problem 2

In [209]:
useducation_df_exp = useducation_df_orignal[['INSTRUCTION_EXPENDITURE','SUPPORT_SERVICES_EXPENDITURE','OTHER_EXPENDITURE','CAPITAL_OUTLAY_EXPENDITURE','Scoring Variable']].copy()
useducation_df_exp.corr().loc['Scoring Variable']

INSTRUCTION_EXPENDITURE         0.107379
SUPPORT_SERVICES_EXPENDITURE    0.103285
OTHER_EXPENDITURE               0.002105
CAPITAL_OUTLAY_EXPENDITURE      0.040287
Scoring Variable                1.000000
Name: Scoring Variable, dtype: float64

It seems Instruction Expenditure is the most correlated expenditure to the scoring variable.

## Problem 3

In [210]:
X = StandardScaler().fit_transform(useducation_df_exp[['INSTRUCTION_EXPENDITURE','SUPPORT_SERVICES_EXPENDITURE','OTHER_EXPENDITURE','CAPITAL_OUTLAY_EXPENDITURE']])

sklearn_pca = PCA(n_components = 1)
PCA_Val = sklearn_pca.fit_transform(X)


In [211]:
print(
    'The percentage of total variance in the dataset explained by each',
    'component from Sklearn PCA.\n',
    sklearn_pca.explained_variance_ratio_
)

The percentage of total variance in the dataset explained by each component from Sklearn PCA.
 [0.94215574]


The first component explains roughly 94.2% of the total variance.

## Problem 4

In [214]:
useducation_df_orignal.insert(26,'PCA Value', PCA_Val)
useducation_df_orignal[['INSTRUCTION_EXPENDITURE','SUPPORT_SERVICES_EXPENDITURE','OTHER_EXPENDITURE','CAPITAL_OUTLAY_EXPENDITURE','TOTAL_EXPENDITURE','PCA Value','Scoring Variable']].corr()

,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,TOTAL_EXPENDITURE,PCA Value,Scoring Variable
INSTRUCTION_EXPENDITURE,1.000000,0.977676,0.898960,0.894339,0.991188,0.971607,0.107379
SUPPORT_SERVICES_EXPENDITURE,0.977676,1.000000,0.941371,0.918926,0.993905,0.988810,0.103285
OTHER_EXPENDITURE,0.898960,0.941371,1.000000,0.904904,0.934352,0.964577,0.002105
CAPITAL_OUTLAY_EXPENDITURE,0.894339,0.918926,0.904904,1.000000,0.932103,0.957313,0.040287
TOTAL_EXPENDITURE,0.991188,0.993905,0.934352,0.932103,1.000000,0.992245,0.098813
PCA Value,0.971607,0.988810,0.964577,0.957313,0.992245,1.000000,0.065556
Scoring Variable,0.107379,0.103285,0.002105,0.040287,0.098813,0.065556,1.000000


The correlation is roughly 6.6%.

## Problem 5

The expenditure variables seem to be the better choice because they are more correlated. The expenditure variables are so connected that it can cause instability due to the correlations being greater 0.7 or 0.8.